<a href="https://colab.research.google.com/github/pvai-umich/SIADS697/blob/main/Data/Load_Kaggle_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

This notebook will contain the data loading scripts for the two kaggle data sets.

Set up the Kaggle login and API key:

In [1]:
import os
import zipfile
import pandas as pd

#comment

os.environ['KAGGLE_USERNAME'] = "pvaiciunas" # username from the json file
os.environ['KAGGLE_KEY'] = "fba13e65a047582a90e611238731617e" # key from the json file

## Kaggle Indeed Jobs Dataset

Source:
https://www.kaggle.com/promptcloud/indeed-job-posting-dataset

The next code block will download the zip file, unzip, and read the csv file into a pandas dataframe.

In [2]:
!kaggle datasets download -d promptcloud/indeed-job-posting-dataset # api copied from kaggle

with zipfile.ZipFile('indeed-job-posting-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

dat_indeed = pd.read_csv("home/sdf/marketing_sample_for_trulia_com-real_estate__20190901_20191031__30k_data.csv")


 94% 45.0M/48.0M [00:00<00:00, 68.7MB/s]
100% 48.0M/48.0M [00:00<00:00, 80.4MB/s]


## Kaggle Monster Jobs Dataset

Source:
https://www.kaggle.com/PromptCloudHQ/us-jobs-on-monstercom

The next code block will download the zip file, unzip, and read the csv file into a pandas dataframe.

In [3]:
!kaggle datasets download -d PromptCloudHQ/us-jobs-on-monstercom

with zipfile.ZipFile('us-jobs-on-monstercom.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

dat_monster = pd.read_csv("monster_com-job_sample.csv")

 89% 17.0M/19.0M [00:00<00:00, 37.6MB/s]
100% 19.0M/19.0M [00:00<00:00, 35.2MB/s]


# Clean and Combine the Data


####Initial Thoughts

Ideally we'd like to combine the two data sets. Both have a job description that we can tokenize pretty easily. However, the challenge becomes in creating usable target variables. 

There is some sector information. So ideally we'd like the target variable to be a combination of the two. We could have two different models, one for sector, and one for job title, or make it one model for 'sector - job title'. However, the sector/industry data isn't present for Indeed like it is for Monster.

Getting both datasets on the same page though will be difficult.

#### Quick exploration code


In [28]:
## Indeed Data
#dat_indeed.head()
#dat_indeed.columns
#dat_indeed['Job Description'][0] # Source for features
dat_indeed['Job Title'].value_counts()[0:50] # source for target variable

## Monster Data
#dat_monster.head()
#dat_monster.columns
#dat_monster['job_description'][0] # Source for features
#dat_monster['job_title'].value_counts()[0:50] # Source for target variable
#dat_monster['sector'].value_counts() # Source for target variable
#dat_monster['organization'].value_counts() # Source for target variable



Account Executive                       329
Sales Associate/Beauty Advisor          209
Account Manager                         192
General Manager                         180
Sales Associate, Retail Part Time       148
Lids Assistant Manager Full-Time        128
Customer Service Representative         124
Store Manager                           120
Lids Store Manager                      119
Lids District Mgr in Training (DMIT)    112
Business Development Representative     107
Leasing Consultant                      107
Retail Sales Teammate                   102
Assistant Store Manager Trainee         102
Sales Development Representative         94
Sales Representative                     88
Sales Associate                          85
Management Trainee                       82
Enterprise Account Executive             80
Business Development Manager             74
Assistant Manager                        72
Product Manager                          71
Marketing Manager               

Create some graphs to show the prevalence of senior vs junior, as well as the concentrated nature of job types in the Indeed data

In [36]:

import seaborn as sns

jobType_data = {'Title' : ['Sales',
                             'Business Dev.',
                             'Retail',
                             'Finance',
                             'Graphic Design',
                             'Oil'],
                  'Count' : [len(dat_indeed[dat_indeed['Job Title'].str.contains('Sales')]),
                             len(dat_indeed[dat_indeed['Job Title'].str.contains('Business Development')]),
                             len(dat_indeed[dat_indeed['Job Title'].str.contains('Retail')]),
                             len(dat_indeed[dat_indeed['Job Title'].str.contains('Finan')]),
                             len(dat_indeed[dat_indeed['Job Title'].str.contains('Graphic')]),
                             len(dat_indeed[dat_indeed['Job Title'].str.contains('Oil')])]}
jobType_df = pd.DataFrame(data = jobType_data)

seniority_data = {'Title' : ['Senior / Executive',
                             'Manager / Director',
                             'Junior / Associate'],                             
                  'Count' : [len(dat_indeed[dat_indeed['Job Title'].str.contains('Senior|Executive')]),
                             len(dat_indeed[dat_indeed['Job Title'].str.contains('Manager|Director')]),
                             len(dat_indeed[dat_indeed['Job Title'].str.contains('Junior|Associate|Trainee')])]}




seniority_df = pd.DataFrame(data = seniority_data)

seniority_df

,Title,Count
0,Senior / Executive,3592
1,Manager / Director,9206
2,Junior / Associate,2377


## Clean Indeed Data

'Job Description' holds the main text we'll need for creating features. We can use gensim to tokenize the job description as well as
add some common bigrams. Alternatively we can use TFID.

The bigger challenge will be getting the target variable in a 


#### Creating Target Variable

There are many similar titles but have an extra word or something that doesn't let it be combined with others. Will do some manual cleaning to get as much overlap between titles as possible. This is being done iteratively and manually.

In [19]:
# Create another version of the dat_indeed df so we can compare at the end
dat_indeed = dat_indeed.copy()

# manually identify areas where we can clean titles to get things more homogenous
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Lids ','')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Senior ','')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Junior ','')

# Do some more specific cleaning of titles 
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace(', Retail Part Time','')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Sales Associate/Beauty Advisor','Beauty Advisor')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('BEAUTY CONSULTANT','Beauty Advisor')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace(' I Store Operations','')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace(' - Prospecting - Financial Solutions','')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('General Manager II','General Manager')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Entry Level Business Development Trainee – Traveling','Business Development')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('District Mgr in Training (DMIT)','District Manager')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('HERO Assistant Store Manager Trainee (Military and Veterans)','Assistant Store Manager')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Business Development Representative','Business Development')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Enterprise Sales Account Executive','Enterprise Account Executive')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('STORE MANAGER','Store Manager')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace("Macy's Locker Room Assistant Manager Full-Time",'Assistant Manager')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('General Manager Full Time-GMA-203100','General Manager')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Business Development Associate','Business Development')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Merrill Lynch Financial Advisor Business Development Program','Financial Analyst')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Business Development Associate','Business Development')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Marketing Assistant','Marketing Associate')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace("Macy's Locker Room Store Manager",'Store Manager')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Assistant Manager Full-Time','Assistant Manager')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Sales Development Representative','Sales Representative')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Enterprise Sales Executive','Sales Executive')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Senior Account Executive','Account Executive')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Sales Development Representative','Sales Representative')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Inside Sales Representative','Sales Representative')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Sales Representative','Sales Associate')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Restaurant Associate-Horizon','Restaurant Associate')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Beauty Consultant','Beauty Advisor')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Associate Financial Advisor Development Program (AFADP)','Financial Advisor')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Senior Financial Analyst','Financial Analyst')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Store Manager in Training','Store Manager')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Regional Sales Director','Sales Director')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Seasonal Sales Associate','Sales Associate')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Office Administrator','Administrator')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Sales Associate (PT)','Sales Associate')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('PT Perishable Associate','Perishable Associate')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('FT Perishable Associate','Perishable Associate')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Business Development Manager','Business Development')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Salesperson','Sales Associate')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Sales Director','Sales Executive')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Purchasing Agent','Buyer')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Director of Sales','Sales Director')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Business Analyst','Financial Analyst')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Staff Accountant','Accountant')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Accounting Manager','Accountant')
dat_indeed2['Job Title'] = dat_indeed2['Job Title'].str.replace('Sr. Financial Analyst','Financial Analyst')
dat_indeed2['Job Title'] = dat_indeed2['Job Title'].str.replace('Finance Manager','Financial Analyst')
dat_indeed2['Job Title'] = dat_indeed2['Job Title'].str.replace('Associate Financial Advisor Development Program (AFADP)','Financial Analyst')



# Want to add some more of these titles if possible
#dat_indeed2['Job Title'] = dat_indeed2['Job Title'].str.replace('','Software Engineer')
#dat_indeed2['Job Title'] = dat_indeed2['Job Title'].str.replace('','Financial Analyst')
#dat_indeed2['Job Title'] = dat_indeed2['Job Title'].str.replace('','Staff Accountant')
#dat_indeed2['Job Title'] = dat_indeed2['Job Title'].str.replace('','Marketing Associate')
#dat_indeed2['Job Title'] = dat_indeed2['Job Title'].str.replace('','Graphic Designer')


# And a whole section on data-related ones so that we can include 'data scientist' in the results
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Finance Data Analyst','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Marketing Data Analyst','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Business Data Analyst','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Cloud Data Architect','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Azure Data Architect ','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Lead Data Scientist','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Head of Data','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('AWS Data Architect','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Data Engineer II','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('SAP Data Architect (MDG, BODS) - Manager','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Data Entry Clerk','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Sr Data Architect - Data Warehouse & MPP - Nationwide Opportunities','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('SAP Data Architect (MDG, BODS) - Specialist Leader','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Sr Data Architect - Data Lake & Analytics - Nationwide Opportunities','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Data Strategy Architect - Cloud, Big Data, Analytics, ML/AI/Data Science','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Quant Specialist Sales (Enterprise Data Sales)','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('SAP Data Scientist (MDG, BODS) - Consultant','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Big Data Scientist','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Data Analyst','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Data Architect','Data Scientist')
dat_indeed['Job Title'] = dat_indeed['Job Title'].str.replace('Data Engineer','Data Scientist')




#dat_indeed['Job Title'].value_counts()[0:50] # source for target variable
#dat_indeed2['Job Title'].value_counts()[51:110] # source for target variable


# Check which job titles left have 'Data' in them
#x = dat_indeed2[dat_indeed2['Job Title'].str.contains("Data")]
#x['Job Title'].value_counts()[0:50]



In [27]:
dat_indeed[dat_indeed['Job Title'].str.contains("Graphic")]['Job Title'].value_counts()[0:50]


Graphic Designer                                                               28
Retail Signs & Graphics Business Development Representative                     3
Junior Graphic Designer                                                         3
Senior Graphic Designer                                                         2
Graphic Designer / Production Specialist                                        1
Marketing and Graphics Specialist                                               1
Graphic Designer Publishing                                                     1
Graphic Designer · Part-Time                                                    1
Graphic Production Artist                                                       1
Motion Graphics Specialist                                                      1
Graphic Design Internship (Fall 2019)                                           1
Graphic Designer and Print Production                                           1
Marketing - Grap

#### Creating Features

In [ ]:

from gensim.parsing.preprocessing import preprocess_string, STOPWORDS

# Quick default tokenization of the job description
dat_indeed['tokens'] = dat_indeed['Job Description'].apply(lambda x: preprocess_string(x))
# Remove the stopwords
dat_indeed['tokens'] = dat_indeed['tokens'].apply(lambda x: [words for words in x if not words in STOPWORDS])


Experienced (Non-Manager)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

## Clean Monster Data

The job-titles here have a lot of weird stuff going on. This includes have 'job' included almost all the time, and it also specifies the location. We'll need to strip both out so that we can get a better job title overlap

#### Creating Target Variable


In [ ]:
dat_monster['job_title'].value_counts()[0:50] # Source for target variable
#dat_monster['sector'].value_counts() # Source for target variable
#dat_monster['organization'].value_counts() # Source for target variable

Monster                                                                                                                                 318
Shift Supervisor Job in Camphill                                                                                                        256
RN                                                                                                                                       70
Shift Supervisor - Part-Time Job in Camphill                                                                                             56
Manager                                                                                                                                  50
Please apply only if you are qualified.                                                                                                  31
LEAD SALES ASSOCIATE-FT Job in Columbus                                                                                                  26
ASST STORE MGR Job i

### Creating Features

In [ ]:
# Quick default tokenization of the job description
dat_monster['tokens'] = dat_monster['job_description'].apply(lambda x: preprocess_string(x))
# Remove the stopwords
dat_monster['tokens'] = dat_monster['tokens'].apply(lambda x: [words for words in x if not words in STOPWORDS])


## Combine the Data